# Prepare Web Driver

In [202]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome('driver/chromedriver.exe')


C:\Users\Asus\AppData\Local\Temp\ipykernel_15112\1150006410.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('driver/chromedriver.exe')


# https://id.indeed.com/

In [203]:
province_list = [
    'aceh', 'sumatra utara', 'sumatra barat', 'riau', 'jambi',
    'kepulauan riau', 'bengkulu', 'sumatra selatan', 'kepulauan bangka belitung', 'lampung',
    'banten', 'jakarta', 'jawa barat', 'jawa tengah', 'yogyakarta', 
    'jawa timur', 'bali', 'nusa tenggara barat', 'nusa tenggara timur', 'kalimantan barat',
    'kalimantan tengah', 'kalimantan selatan', 'kalimantan timur', 'kalimantan utara', 'sulawesi barat',
    'sulawesi selatan', 'sulawesi tenggara', 'gorontalo', 'sulawesi utara',
    'maluku utara', 'maluku', 'papua barat', 'papua'
    ]

base_url = 'https://id.indeed.com/'
loker_url = base_url + 'lowongan-kerja'
prov = province_list[0]
loker_prov_url = loker_url + '?l=' + province_list[0]
driver.get(loker_prov_url)


In [204]:
job_html = driver.find_elements(
    by=By.CSS_SELECTOR, value='.slider_container')

# job_name_list = []
# company_name_list = []
# company_location_list =  []
# salary_min_list = []
# salary_max_list = []
# job_type = []
# job_desk = []

df = pd.DataFrame({
    'job_name': [],
    'company_name': [],
    'company_location': [],
    'salary_min': [],
    'salary_max': [],
    'job_type': [],
    'job_desc': []
})

for jh in job_html:
    jh.click()

    detail_job_url = driver.find_element(by=By.CSS_SELECTOR, value='HTML').find_element(
        by=By.CSS_SELECTOR, value='iframe#vjs-container-iframe').get_attribute('src')

    driver.execute_script(f'''window.open("{detail_job_url}","_blank");''')

    driver.switch_to.window(driver.window_handles[1])

    job_name = driver.find_element(by=By.CSS_SELECTOR, value='h1.jobsearch-JobInfoHeader-title').text
    job_name = job_name.replace('\n- job post', '')

    company_info_html = driver.find_element(by=By.CSS_SELECTOR, value='.jobsearch-JobInfoHeader-subtitle')
    company_name = company_info_html.find_element(by=By.CSS_SELECTOR, value='div').text
    company_location = company_info_html.find_element(by=By.XPATH, value='div[2]').text

    jobdesc_0 = driver.find_elements(by=By.CSS_SELECTOR,
                    value='.jobsearch-JobDescriptionSection-sectionItem')

    salary_min, salary_max = -1, -1
    job_type = ''
    for jd in jobdesc_0:

        jd = jobdesc_0[0].text.split('\n')
        if jd[0] == 'Salary':
            salary = jd[1]
            salary = salary.replace('Salary\n', '')
            salary = salary.replace('Rp. ', '')
            salary = salary.replace(' per bulan', '')
            salary = salary.replace('.', '')
            salary = salary.split(' - ')
            salary_min = int(salary[0])
            salary_max = int(salary[1])
        if jd[0] == 'Job Type':
            job_type = jd[1]

    job_desc = driver.find_element(by=By.CSS_SELECTOR,
                                  value='#jobDescriptionText').text

    df_temp = pd.DataFrame({
        'job_name': [job_name],
        'company_name': [company_name],
        'company_location': [company_location],
        'salary_min': [salary_min],
        'salary_max': [salary_max],
        'job_type': [job_type],
        'job_desc': [job_desc]
    })
    df = pd.concat([df, df_temp], ignore_index=True)

    driver.close()
    driver.switch_to.window(driver.window_handles[0])

df.to_json('loker_scraping.json', DataFrame='record')
df


,job_name,company_name,company_location,salary_min,salary_max,job_type,job_desc
0,Store Manager,D'cost Group,Banda Aceh,7000000.0,7500000.0,,Gambaran Pekerjaan:\nMelakukan peningkatan pen...
1,Customer Service Development Program (Aceh),PT Bank Muamalat Indonesia,Banda Aceh,-1.0,-1.0,Full-time,Customer Service Development Program (Aceh)\nP...
2,Agent Konsultan,Yayasan Aupair Indonesia Jerman,Banda Aceh,5000000.0,8000000.0,,Agent Konsultan YAIJ adalah seseorang yang bek...
3,Surveyor/Field Officer,PT Magada Tirta Amerta,Banda Aceh,-1.0,-1.0,Full-time,"Hi, Klien kami (Multinational Online Transport..."
4,Ground Revenue - Aceh (Contract Based),OYO Hotels Indonesia,Banda Aceh,-1.0,-1.0,Full-time,Responsibilities:\nMaking sure all the booking...
5,Supervisor Consumer Engagement - Lhokseumawe L...,"Philip Morris International\n1,716 reviews",Lhokseumawe,-1.0,-1.0,Full-time,"Be a part of a revolutionary change\n\nAt PMI,..."
6,Sales Executive,PT Sinergi Tridaya Medical,Banda Aceh,5000000.0,5500000.0,,Tugas dan Tanggung Jawab:\n1. Detailing dan me...
7,Service Advisor (Aceh),PT CSM Corporatama Indorent,Banda Aceh,4500000.0,5000000.0,,Kaulifikasi :\nPria usia maks. 35 tahun\nPendi...
8,Lead ops Agent - Aceh,"DHL Express\n22,073 reviews",Banda Aceh,-1.0,-1.0,Contract,(22003F3)\nIndonesia-Aceh-Banda Aceh | Full-ti...
9,Field Collection,OTO Group,Langsa,-1.0,-1.0,,Menjaga account dan memberikan solusi dalam pe...


In [207]:
df.to_json('loker_scraping.json', 'table')

In [209]:
driver.quit()

KeyboardInterrupt: 

In [ ]:
slider_html = driver.find_elements(
    by=By.CSS_SELECTOR, value='.slider_container')


In [ ]:
slider_html[0].click()


In [ ]:
detail_job_url = driver.find_element(by=By.CSS_SELECTOR, value='HTML').find_element(
    by=By.CSS_SELECTOR, value='iframe#vjs-container-iframe').get_attribute('src')


In [ ]:
driver.execute_script(f'''window.open("{detail_job_url}","_blank");''')


In [ ]:
driver.switch_to.window(driver.window_handles[1])


In [ ]:
job_name = driver.find_element(by=By.CSS_SELECTOR, value='h1.jobsearch-JobInfoHeader-title').text
job_name = job_name.replace('\n- job post', '')
job_name


In [ ]:
company_info_html = driver.find_element(by=By.CSS_SELECTOR, value='.jobsearch-JobInfoHeader-subtitle')
company_name = company_info_html.find_element(by=By.CSS_SELECTOR, value='div').text
company_location = company_info_html.find_element(by=By.XPATH, value='div[2]').text

print(f'{company_name} - {company_location}') 

In [ ]:
salary = driver.find_elements(by=By.CSS_SELECTOR,
                              value='.jobsearch-JobDescriptionSection-sectionItem')
salary


In [ ]:
salary[0].text.split('\n')

In [ ]:
salary = driver.find_elements(by=By.CSS_SELECTOR,
                    value='.jobsearch-JobDescriptionSection-sectionItem').text
salary = salary.replace('Salary\n', '')
salary = salary.replace('Rp. ', '')
salary = salary.replace(' per bulan', '')
salary = salary.replace('.', '')
salary = salary.split(' - ')
salary_min = int(salary[0])
salary_max = int(salary[1])

print(f'''
salary min = {salary_min}
salary max = {salary_max}
''')



In [ ]:
job_type = driver.find_element(by=By.XPATH,
                    value='//*[@id="jobDetailsSection"]/div[3]/div[2]').text
job_type

In [ ]:
jobdesc = driver.find_element(by=By.CSS_SELECTOR,
                    value='#jobDescriptionText').text
print(jobdesc)

In [ ]:
driver.close()
driver.switch_to.window(driver.window_handles[0])

In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
driver.delete_all_cookies()
driver.refresh()
driver.quit()
